<DIV ALIGN=CENTER>

# Introduction to NLP: Semantic Analysis
## Professor Robert J. Brunner
  
</DIV>  
-----
-----


## Introduction

Semantic Analysis
Derive meaning from words, chunks, tokens.

word2vec, doc2vec

gensim

-----



-----

## Student Activity

In the preceding cells, we . Now that you have run the Notebook,
go back and make the following changes to see how the results change.

1. Change 
2. Try 
3. Can 

-----

In [1]:
nltk:
    concordance
    similar
    common_contexts
    
    dispersion_plot

SyntaxError: invalid syntax (<ipython-input-1-16fc9134832a>, line 1)

In [4]:
import gensim

#model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

sentences = [['Some text', 'more text'], ['New row', 'something else']]

model = gensim.models.Word2Vec(sentences, min_count=1)